### **6.4 - Dependencies and Helpers**

Vamos a jugar un poco con las dependencias porque hay pocos detalles que conocer sobre ellas y hay algunas funciones bastante útiles que quizás no conozcas. Así que vamos a empezar. Empecemos con algunos conceptos básicos. En primer lugar, hay dos maneras de definir las dependencias en Airflow. Está la forma antigua que nunca usarás y la nueva, que ya no es realmente nueva. Así, la forma antigua es mediante el uso de las funciones "set_upstream" y "set_downstream". Así que básicamente quieres decir que "t1" debe ejecutarse primero y luego "t2", entonces puedes decir "t1.set_downstream(t2)". O puedes decir que "t2" tiene la tarea upstream "t1" => "t2.set_upstream(t1)". 

<center><img src="https://i.postimg.cc/gj5dZDDp/a1354.png"></center>

Esta es la forma antigua de definir las dependencias. La nueva forma, que tampoco es realmente nueva, consiste en utilizar los "operadores bit shift". Puedes decir que "t1" es la tarea upstream de "t2" o puedes decir que "t2" es la tarea downstream de "t1" con el operador bit shift correcto. 

<center><img src="https://i.postimg.cc/CM6wXhV2/a1355.png"></center>

Esto es lo básico. Ahora imaginemos que queremos decir que [t1, t2, t3] son las tareas upstream de "t5". ¿Puedes hacerlo? Claro que sí.

<center><img src="https://i.postimg.cc/rF0qLBVw/a1356.png"></center>

Pero, ¿y si quieres decir que [t4, t5, t6] son ahora las tareas downstream de las tareas upstream [t1, t2, t3]. ¿Funciona? Pues no, porque Airflow no sabe exactamente lo que quieres, es decir, si quieres que "t4" dependa de "t1, t2 y t3" o si quieres que "t4" dependa de "t1". Es decir, no lo sabe, así que no eres capaz de crear dependencias entre listas como esa. 

<center><img src="https://i.postimg.cc/vZ5M456R/a1357.png"></center>

Si vas a la interfaz de usuario, obtendrás el siguiente error diciendo que no puedes crear dependencias entre dos listas. 

<center><img src="https://i.postimg.cc/15ksRvXc/a1358.png"></center>

¿Qué más puedes hacer? Bueno, si vuelves a tu editor de código, ¿qué pasa si quieres crear dependencias cruzadas? Por ejemplo, quieres decir que "t4" depende de "t1, t2 y t3", "t5" depende de "t1, t2 y t3" y "t6" depende de "t1, t2 y t3". Una forma es escribiendo:

<center><img src="https://i.postimg.cc/rpJkJcJQ/a1359.png"></center>

Pero de nuevo, esta no es la forma más fácil y mejor de hacerlo. Para crear dependencias cruzadas entre dos listas de tareas, una forma es utilizando una función súper útil llamada "cross_downstream". En primer lugar, es necesario importar esa función, así que escribe "from airflow.models.baseoperator import cross_downstream".

<center><img src="https://i.postimg.cc/YSk7Yj1v/a1360.png"></center>

Y una vez que tengas esta función, sólo tienes que decir:

<center><img src="https://i.postimg.cc/85Xg2Hxv/a1361.png"></center>

Si guardas el archivo y echas un vistazo a la interfaz de usuario de Airflow, vamos a actualizar la página y haz clic en "dependency" y en Graph view y obtendrás las siguientes dependencias cruzadas.

<center><img src="https://i.postimg.cc/Px5kVcVh/a1362.png"></center>
<center><img src="https://i.postimg.cc/zvLY8h1c/a1363.png"></center>

Así puedes ver que "t4" depende de "t1, t2, t3" y "t5" depende de "t1, t2, t3" y lo mismo para "t6". 

<center><img src="https://i.postimg.cc/wMS8vBqT/a1364.png"></center>
<center><img src="https://i.postimg.cc/HWKC36wD/a1365.png"></center>

Es realmente potente. Una cosa que hay que saber sobre esta función es que no devuelve nada. Así que si tienes otra tarea, por ejemplo, "t7" y quiero decir que "t4, t5, t6" son tareas upstream de "t7", no puedes hacer eso. Si intentas hacerlo, recibirás un error porque "cross_downstream" no devuelve nada.

<center><img src="https://i.postimg.cc/SNMbPcdX/a1366.png"></center>

Si quieres definir esas dependencias, tienes que escribir "t4, t5 y t6" en otra línea, y luego decir que son tareas upstream de "t7". 

<center><img src="https://i.postimg.cc/YCBBBjDd/a1367.png"></center>

Ten esto en cuenta porque es muy fácil acabar con ese problema cuando utilizas "cross_downstream". Ahora, hay otra función súper útil llamada "chain", y con "chain" puedes crear dependencias más complejas. Veámoslo. Así que el primer paso es importar esa función:

<center><img src="https://i.postimg.cc/9QP5mVvC/a1368.png"></center>

Otra pregunta es, ¿cómo puedes crear dependencias en cadena (chain dependencies)? Imaginemos que quieres tener "t1" primero y luego quieres ejecutar "t2" y "t3" en la misma línea, así que quieres tener "t2" depende de "t1", y "t3" depende de "t1" y luego quieres tener "t5" depende de "t3", y "t4" depende de "t2". ¿Cómo se puede representar esto con "chain"? Bueno, una cosa que puedes hacer es usar esa función, así que escribes

<center><img src="https://i.postimg.cc/BvqWtk0D/a1369.png"></center>

y tienes que asegurarte de que la otra lista junto a la primera lista tiene el mismo tamaño de la primera lista, así que puedes escribir:

<center><img src="https://i.postimg.cc/XvxR5LjP/a1370.png"></center>

Si esa lista tiene un tamaño diferente de esa lista, no funcionará. Ok, entonces asegúrate de que tu lista tiene el mismo tamaño y entonces puedes decir "t6" como la última tarea. 

<center><img src="https://i.postimg.cc/63mFjQGL/a1371.png"></center>

Si guardas el archivo y echas un vistazo a la interfaz de usuario de Airflow obtendrás las siguientes dependencias. Bonito, ¿verdad? 

<center><img src="https://i.postimg.cc/XJJTk5XY/a1372.png"></center>

Sí, pero puedes hacerlo mejor. ¿Qué pasa si quieres mezclar "cross_downstream" y "chain"? Pues sólo tienes que escribir:

<center><img src="https://i.postimg.cc/2yLMPfQp/a1373.png"></center>

Guarda el archivo, echa un vistazo a la interfaz de usuario de Airflow, actualiza la página y ahora obtendrás las siguientes dependencias en las que "t4" depende de "t3 y t2"

<center><img src="https://i.postimg.cc/hhF6n3sX/a1374.png"></center>

y "t5" depende de "t2 y t3". 

<center><img src="https://i.postimg.cc/NGy0g9bt/a1375.png"></center>

No dudes en jugar con estas funciones ya que son capaces de crear dependencias bastante complejas y no son tan conocidas. Ahora puedes crear data pipelines más complejos gracias a estas funciones.